In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time
import pytz

# -----------------------------
# TIME RESTRICTION (6 PM – 9 PM IST)
# -----------------------------
ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist).time()

if not (time(18, 0) <= current_time <= time(21, 0)):
    print("Graph is available only between 6 PM IST and 9 PM IST.")
else:

    # -----------------------------
    # LOAD DATA (Assume CSV file)
    # -----------------------------
    df = pd.read_csv("app_data.csv")

    # Convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    # -----------------------------
    # APPLY FILTER CONDITIONS
    # -----------------------------
    df = df[
        (df['reviews'] > 500) &
        (~df['app_name'].str.lower().str.startswith(('x', 'y', 'z'))) &
        (~df['app_name'].str.contains('s', case=False)) &
        (df['category'].str.upper().str.startswith(('E', 'C', 'B')))
    ]

    # -----------------------------
    # TRANSLATE CATEGORIES
    # -----------------------------
    category_translation = {
        "Beauty": "सौंदर्य",        # Hindi
        "Business": "வணிகம்",      # Tamil
        "Dating": "Partnersuche"   # German
    }

    df['category'] = df['category'].replace(category_translation)

    # -----------------------------
    # MONTHLY AGGREGATION
    # -----------------------------
    df['month'] = df['date'].dt.to_period('M')
    monthly_data = df.groupby(['month', 'category'])['installs'].sum().reset_index()
    monthly_data['month'] = monthly_data['month'].dt.to_timestamp()

    # -----------------------------
    # PLOT TIME SERIES
    # -----------------------------
    plt.figure(figsize=(12, 6))

    for category in monthly_data['category'].unique():
        cat_data = monthly_data[monthly_data['category'] == category].sort_values('month')

        # Calculate MoM Growth
        cat_data['growth'] = cat_data['installs'].pct_change()

        # Plot Line
        plt.plot(cat_data['month'], cat_data['installs'], label=category)

        # Highlight Growth > 20%
        significant_growth = cat_data['growth'] > 0.20

        plt.fill_between(
            cat_data['month'],
            cat_data['installs'],
            where=significant_growth,
            alpha=0.3
        )

    # -----------------------------
    # GRAPH SETTINGS
    # -----------------------------
    plt.title("Total Installs Trend Over Time (Filtered Categories)")
    plt.xlabel("Month")
    plt.ylabel("Total Installs")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Graph is available only between 6 PM IST and 9 PM IST.
